# HW3: kNN, Naive Bayes and Text Feature Selection

For this HW, we'll explore kNN and Naive Bayes further and look at different feature selection strategies. First of all, download the dataset for this HW from <a href="https://raw.githubusercontent.com/biddata/datascience/master/F15/hw4/data.tar.gz">here</a> and unpack into a hw4 directory with 
<pre>
tar xvf hw4data.tar.gz
</pre>
The dataset is from the RCV1 v2 corpus which is a collection of news articles and category labels. There are 103 categories, and each document can lie in one or more categories. 

The sparse train and test matrices are encoded as nnz x 3 integer matrices, where nnz is the total number of words in all docs. Each row of these matrices contains (doc, word, count) triples. We will store those in sparse matrices whose dimensions are ndocs x nwords


In [ ]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline

Now read the data files. 

In [ ]:
iwords = np.loadtxt("data/words.imat.txt")          # training data matrix in nnz x 3 form - rows are (doc, word, count) triples
cats = np.loadtxt("data/cats.imat.txt")             # training labels in an ndocs x ncats matrix
tiwords = np.loadtxt("data/testwords.imat.txt")     # test data matrix in nnz x 3 form
tcats = np.loadtxt("data/testcats.imat.txt")        # testing labels in an ndocs x ncats matrix

iwords and tiwords encode the sparse train and test matrices. Each row of these matrices contains (doc, word, count) triples. We will store those in sparse matrices whose dimensions are ndocs x nwords. The exact matrix type is CSR which stands for compressed sparse rows. Look up its description here: https://en.wikipedia.org/wiki/Sparse_matrix

In [ ]:
fwords = iwords[:,2]
train = sp.csr_matrix((fwords, (iwords[:,1], iwords[:,0])))
ndocs = train.shape[0]
nwords = train.shape[1]

test = sp.csr_matrix((tiwords[:,2], (tiwords[:,1], tiwords[:,0])),shape=(4000,nwords))  # need to match the number of cols (words)
ntdocs = test.shape[0]

Although the dataset has 103 category labels, we'll focus on training a single category. We will use category 6, which is fairly evenly balanced between positives and negatives. 

In [ ]:
cat6 = cats[:,6]
tcat6 = tcats[:,6]

This time we'll train Scikit-Learn's builtin kNN classifier. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knnc = KNeighborsClassifier(n_neighbors=3)

And then "fit" it to the data. Notice how fast this is! Of course, training kNN is basically a no-op, and predicting is where all the work lies:

In [ ]:
knnc.fit(train, cat6)

Now lets try predicting. This will actually take some time, but hopefully only a few tens of seconds...

In [ ]:
preds = knnc.predict(test);preds

Now we can compute the accuracy on the actual test set labels. 

In [ ]:
np.mean(tcat6 == preds)

Not bad, but kNN is very sensitive to the distance function as we argued in lab. Lets try first normalizing each row of "train" and "test" by the L2-norm of that row, i.e. we will divide by the sqrt of the sum of the squared word counts in the row. Comparing the normalized documents should do much better. 

> TODO: implement row normalization below. Compute the row norms first. Your implementation strategy is important for this to run in reasonable time. **Dont** try to construct an ndocs x ndocs dense matrix. You probably also dont want to iterate over all the docs or all the words. The fastest solution is to pre-multiply by a sparse diagonal matrix which contains the scale factors. No loops are necessary. Normalization should take a fraction of a second. 

In [ ]:
trainnorm =                 # L2 norms for the rows of train
testnorm =                  # L2 norms for the rows of test
trainfact =                 # train correction factor (inverse of trainnorm)
testfact =                  # test correction factor (inverse of testnorm)
trainnmat =                 # scaled (normalized) training mat
testnmat =                  # scaled (normalized) test mat
testnmat.shape              # should be (10000,44718)

Now train and evaluate a knn model using the normalized datasets:

In [ ]:
knnc.fit(trainnmat, cat6)
preds = knnc.predict(testnmat);preds
np.mean(tcat6 == preds)     

> TODO: What was the accuracy of your kNN classifier on the normalized data?

## Naive Bayes Classification

Lets create a Multinomial Naive Bayes classifier. The multinomial distribution models each word in a document as being drawn from a category-specific probability distribution over the vocabulary. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

Now we can fit the NB model, which simply means tabulating the word counts for each label.

In [ ]:
mnb.fit(train, cat6)

...and predict the category labels for cat6:

In [ ]:
preds = mnb.predict(test);preds

> TODO: Lets check the accuracy below. How did NB do compared to kNN? 

In [ ]:
np.mean(tcat6 == preds)

## Frequency-Based Feature Selection

We argued that frequency-based feature selection is useful approach to culling features to reduce the size of a model. In some cases, accuracy improves as well. 

Lets construct a matrix rcounts which contains the counts of features from the training set. 

In [ ]:
rcounts = (train>0).sum(0)
rcounts

Next we'll find the indices of non-zero elements of the predicate (rcounts >= count). These are the features that occurred at least count times. ii is the indices of these features.

We then extract the columns of the train and test matrices corresponding to those high-count features. The last line tells us how many features we kept.

In [ ]:
threshold = 1
ii0 = np.asarray(np.nonzero(rcounts >= threshold)[1])
ii = ii0.reshape(ii0.size,)

strain = train[:,ii]
stest = test[:,ii]
ii.size

Finally we run training and evaluation on train/test data restricted to those columns to get an accuracy score.

In [ ]:
mnb.fit(strain, cat6)
preds = mnb.predict(stest);preds
np.mean(tcat6 == preds)

> TODO: For threshold counts of 1, 2, 5, 10, 20, tabulate the count, number of features kept, and the accuracy score. 

> TODO: How did frequency-based feature selection affect accuracy? How much was model size (proportional to words kept) reduced? 

## Feature Selection Based on Statistical Tests

Lets try a more sophisticated method to select the (word) features. Since we have count data, the Chi-Squared test is appropriate. Remember that Chi-squared can be used on 2x2 contingency tables to determine if there is an interaction. In this case, we test if the label interacts with the feature. 

In [ ]:
fcounts = np.zeros((4,nwords))           # holds the counts of various combinations of label and feature
expected = np.zeros((4,nwords))          # holds the predicted counts of those same combinations

In [ ]:
mcat6 = cat6.reshape(1,ndocs)
fcounts[0,:] = mcat6 * (train > 0)                                   # label and word both = 1
fcounts[1,:] = (1-mcat6) * (train > 0)                               # label = 0, word = 1
fcounts[2,:] = np.sum(mcat6) * np.ones((1,nwords)) - fcounts[0,:]    # label = 1, word = 0
fcounts[3,:] = np.sum(1-mcat6) * np.ones((1,nwords)) - fcounts[1,:]  # label = 0, word = 0


Review the definition of the CHI-squared test here:
https://en.wikipedia.org/wiki/Chi-squared_test
Under the hull hypothesis that category label doesnt matter, we expect to be able to compute 

In [ ]:
fcounts[0,:]

In [ ]:
allf = np.sum(fcounts,axis=0)
pword0 = (fcounts[2,:] + fcounts[3,:]) / allf    # prob word = 0
pword1 = (fcounts[0,:] + fcounts[1,:]) / allf    # prob word = 1
pcat0 = (fcounts[1,:] + fcounts[3,:]) / allf     # prob label = 0
pcat1 = (fcounts[0,:] + fcounts[2,:]) / allf     # prob label = 1

If label and word occurence are independent, then the expected count should be the product of the total count and the corresponding probabilities:

> TODO: Implement the expected counts here:

In [ ]:
expected[0,:] =   # expected count with label = 1, word = 1
expected[1,:] =   # expected count with label = 0, word = 1
expected[2,:] =   # expected count with label = 1, word = 0
expected[3,:] =   # expected count with label = 0, word = 0

To compute the chi-squared statistic value, we want the Sum of the squared differences between actual and predicted counts, normalized by the expected counts. The following two cells perform that calculation. We have to guard against divide-by-zero so we use a small lower-bound on the denominator.

In [ ]:
diff = fcounts - expected
safeexpected = np.maximum(expected,1e-7)

> TODO: Implement the CHI-squared statistic value here

In [ ]:
chisquaredstat =  # implement the chisquared statistic value here
chisquaredstat.shape

We can then use the chi-squared statistic to choose the best features. The CHI-squared stastistic is a measure of interaction between a word feature and the label. The higher its value, the better that word should be as a predictor of the category. We set a threshold and take the words whose chi-squared statistic is above this value:

In [ ]:
chisqthreshold = 1.5
ii0 = np.asarray(np.nonzero(chisquaredstat > chisqthreshold)[0])
ii = ii0.reshape(ii0.size,);ii.shape

Finally we use the indices of good features selected by the chi-squared filter to train and evaluate the Naive Bayes model.

In [ ]:
strain = train[:,ii]
stest = test[:,ii]
mnb.fit(strain, cat6)
preds = mnb.predict(stest);preds
np.mean(tcat6 == preds)

> TODO: Compare the accuracy of CHI-squared feature selection to the frequency-based selection you did earlier. For each frequency threshold (1,2,5,10,20), adjust the chisquared threshold to retain the smallest number of features that is at least as large as the number of features kept by that frequency threshold. Then compute the accuracy in the last cell for that threhold. Finally tabulate both frequency-based and chi-squared accuracies. That is, make a table whose columns are:
* freq threshold
* number of features retained by this frequency threshold
* chisquared threshold 
* number of features retained by this chisquared threshold (should be >= column 2)
* accuracy of frequency thresholded model
* accuracy of chisquared thresholded model
and whose rows are indexed by the frequency thresholds 1, 2, 5, 10, 20


> TODO: Which method gave better accuracy for a given number of features? 